<a href="https://colab.research.google.com/github/sahniaditya007/Crypto-Bot/blob/main/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install Required Libraries

In [3]:
!pip install numpy pandas transformers torch requests websocket-client ccxt backtrader stable-baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

2. Import Required Libraries

In [4]:
import numpy as np
import pandas as pd
import requests
import torch
from transformers import pipeline
import ccxt
import websocket
import json
import backtrader as bt
from stable_baselines3 import PPO

3. Define Sentiment Analysis Model (FinBERT)

In [6]:
sentiment_pipeline = pipeline("text-classification", model="ProsusAI/finbert")

def get_sentiment(text):
    result = sentiment_pipeline(text)[0]
    return result["label"], result["score"]

Device set to use cpu


4. Fetch Crypto News Data

In [7]:
def fetch_crypto_news():
    url = "https://cryptopanic.com/api/v1/posts/?auth_token=5d3efdf3405c86e637926d16c630b32483628a26&filter=trending"
    response = requests.get(url).json()
    return response["results"]

news_data = fetch_crypto_news()
for news in news_data[:5]:
    print(f"Title: {news['title']} -> Sentiment: {get_sentiment(news['title'])}")

Title: BitBoy Busted: Ben Armstrong Arrested in Florida Over Emails to Judge -> Sentiment: ('negative', 0.8386741876602173)
Title: 1.15 Million XRP in 24 Hours: $3 Ready? -> Sentiment: ('neutral', 0.915492057800293)
Title: SEC Ends Probe Into Immutable With No Action Taken -> Sentiment: ('neutral', 0.7034323811531067)
Title: Ripple Plans U.S. Comeback After SEC Case Ends -> Sentiment: ('neutral', 0.6382911205291748)
Title: XRP ETF Incoming? Analyst Says Ripple’s Legal Win Changes Everything -> Sentiment: ('neutral', 0.911679744720459)


5. Fetch Market Data from Binance

In [8]:
binance = ccxt.binance()

def fetch_market_data(symbol="BTC/USDT"):
    ohlcv = binance.fetch_ohlcv(symbol, timeframe="1h", limit=50)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    return df

market_data = fetch_market_data()
print(market_data.head())

            timestamp      open      high       low     close      volume
0 2025-03-24 14:00:00  87704.47  88765.43  87640.77  87961.90  4043.87489
1 2025-03-24 15:00:00  87961.90  88372.00  87678.06  88350.01  1848.70464
2 2025-03-24 16:00:00  88350.01  88499.98  87690.59  88057.54  1818.05892
3 2025-03-24 17:00:00  88057.53  88486.55  87941.06  88461.88  1471.35010
4 2025-03-24 18:00:00  88461.88  88536.00  88123.11  88464.38   915.48486


6. WebSocket for Real-time Order Book Data

In [ ]:
def on_message(ws, message):
    data = json.loads(message)
    print("Order Book Update:", data)

def on_open(ws):
    subscribe_msg = {
        "method": "SUBSCRIBE",
        "params": ["btcusdt@depth"],
        "id": 1
    }
    ws.send(json.dumps(subscribe_msg))

ws = websocket.WebSocketApp("wss://stream.binance.com:9443/ws/btcusdt@depth", on_message=on_message, on_open=on_open)
ws.run_forever()


Order Book Update: {'result': None, 'id': 1}
Order Book Update: {'e': 'depthUpdate', 'E': 1743001617014, 's': 'BTCUSDT', 'U': 64889649883, 'u': 64889650059, 'b': [['86973.24000000', '1.37768000'], ['86973.02000000', '0.00007000'], ['86972.81000000', '0.00021000'], ['86972.80000000', '0.34818000'], ['86972.79000000', '0.19169000'], ['86972.70000000', '0.00000000'], ['86972.19000000', '0.00014000'], ['86972.18000000', '0.75877000'], ['86971.50000000', '0.04600000'], ['86971.25000000', '0.00000000'], ['86970.96000000', '0.00000000'], ['86970.30000000', '0.00000000'], ['86969.84000000', '0.00000000'], ['86969.83000000', '0.00000000'], ['86969.82000000', '0.00000000'], ['86968.80000000', '0.04600000'], ['86968.01000000', '0.07730000'], ['86966.83000000', '0.00000000'], ['86966.39000000', '0.00000000'], ['86966.38000000', '0.00253000'], ['86965.70000000', '0.09233000'], ['86963.83000000', '0.00000000'], ['86959.87000000', '0.00700000'], ['86956.49000000', '0.00000000'], ['86947.25000000', '0

7. Backtesting Market-Making Strategy

In [ ]:
class MarketMakerStrategy(bt.Strategy):
    def __init__(self):
        self.sma = bt.indicators.SimpleMovingAverage(period=10)

    def next(self):
        if self.data.close[0] > self.sma[0]:
            self.buy()
        elif self.data.close[0] < self.sma[0]:
            self.sell()

cerebro = bt.Cerebro()
data = bt.feeds.PandasData(dataname=market_data)
cerebro.adddata(data)
cerebro.addstrategy(MarketMakerStrategy)
cerebro.run()
cerebro.plot()

8. Reinforcement Learning Model for Market Making

In [ ]:
class MarketMakerEnv(bt.Strategy):
    def __init__(self):
        self.reward = 0

    def next(self):
        if self.data.close[0] > self.data.close[-1]:
            self.reward += 1
        else:
            self.reward -= 1

env = MarketMakerEnv()
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)